In [8]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
train_data = pd.read_csv('CreditScore.csv')

print(train_data.info())

In [10]:
target_variable_name = 'Credit_Score'
train_data[target_variable_name] = train_data[target_variable_name].apply(lambda x: 1 if x == 'Good' else 0)

X = train_data.drop(target_variable_name, axis=1)
y = train_data[target_variable_name]

categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [11]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('to_string', FunctionTransformer(lambda x: x.astype(str))),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

for col in categorical_cols:
    X[col] = X[col].astype(str)

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

model = RandomForestClassifier(random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Monthly_Inhand_Salary',
                                                   'Num_Credit_Card',
                                                   'Interest_Rate',
                                                   'Delay_from_due_date',
                                                   'Total_EMI_per_month']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(h...
                                                   'Annual_Income',
                                                   'Num_Bank_Accounts',
                                                   'Num_of_Loan',
                                                   'Type_of_Loan',
                                                   'Num_of_Delayed_Payment',
                                                   'Changed_Credit_Limit',
                                                   'Num_Credit_Inquiries',
                                                   'Credit_Mix',
                                                   'Outstanding_Debt',
                                                   'Credit_Utilization_Ratio',
                                                   'Credit_History_Age',
                                                   'Payment_of_Min_Amount',
                                                   'Amount_invested_monthly',
                                                   'Payment_Behaviour',
                                                   'Monthly_Balance'])])),
                ('model', RandomForestClassifier(random_state=42))])

In [14]:
y_pred = pipeline.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_valid, y_pred))

Accuracy: 0.8892827513387134
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     10865
           1       0.83      0.49      0.61      2394

    accuracy                           0.89     13259
   macro avg       0.86      0.73      0.77     13259
weighted avg       0.88      0.89      0.88     13259

